## Calculate Grit for bulk and single cell perturbseq data

Also calculate UMAP embeddings for each perturbation at the same time.

In [1]:
import pathlib
import numpy as np
import pandas as pd

from pycytominer import aggregate
from pycytominer.cyto_utils import infer_cp_features

from cytominer_eval import evaluate

import umap

In [2]:
np.random.seed(2021)

In [3]:
gse_id = "GSE132080"
perturbseq_data_dir = pathlib.Path("../../0.download-data/data/perturbseq/")
perturbseq_screen_phenotypes = "paper_supplement/Table_S16_perturb-seq_screen_phenotypes.txt"

In [4]:
# Load finalized single cell perturbseq data
gene_exp_file = pathlib.Path(f"{perturbseq_data_dir}/{gse_id}_final_analytical.tsv.gz")

sc_gene_exp_df = pd.read_csv(gene_exp_file, sep="\t")
gene_features = [x for x in sc_gene_exp_df if not x.startswith("Metadata_")]

print(sc_gene_exp_df.shape)
sc_gene_exp_df.head()

(23537, 1012)


/Users/akalinin/Software/miniconda3/envs/grit-benchmark/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3442: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Metadata_cell_identity,Metadata_cell_barcode,Metadata_guide_identity,Metadata_read_count,Metadata_UMI_count,Metadata_coverage,Metadata_gemgroup,Metadata_good_coverage,Metadata_number_of_cells,Metadata_gene_identity,...,YPEL4,YPEL5,ZBTB38,ZFAS1,ZFP36L1,ZNF365,ZNF43,ZNF483,ZNF556,ZYX
0,sc_profile_0,AAACCTGAGAGTAATC-1,RAN_RAN_+_131356438.23-P1P2_12,544.0,34.0,16.000000,1.0,True,1.0,RAN,...,-0.09309,-0.632468,-0.539091,-1.633805,-0.876912,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
1,sc_profile_1,AAACCTGAGGGATCTG-1,neg_ctrl_non-targeting_00089,267.0,19.0,14.052632,1.0,True,1.0,neg,...,-0.09309,-0.632468,-0.539091,1.610220,1.322265,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
2,sc_profile_2,AAACCTGAGGTCATCT-1,POLR2H_POLR2H_+_184081251.23-P1P2_08,622.0,34.0,18.294118,1.0,True,1.0,POLR2H,...,-0.09309,-0.632468,-0.539091,-0.567252,0.057540,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
3,sc_profile_3,AAACCTGCAATGGAGC-1,TUBB_TUBB_+_30688126.23-P1_03,433.0,20.0,21.650000,1.0,True,1.0,TUBB,...,-0.09309,1.765279,-0.539091,-1.659420,-0.876912,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
4,sc_profile_4,AAACCTGCACCAGGCT-1,CDC23_CDC23_-_137548987.23-P1P2_04,136.0,8.0,17.000000,1.0,True,1.0,CDC23,...,-0.09309,0.686753,-0.539091,0.541260,0.934539,-0.012155,-0.017763,-0.026877,-0.0524,1.406718


In [5]:
print(len(gene_features))

1000


In [6]:
# Load activities results (bulk)
file = perturbseq_data_dir / perturbseq_screen_phenotypes
activity_df = pd.read_csv(file, sep="\t").rename({"Unnamed: 0": "id"}, axis="columns")

# Create a perturbation column to match with other IDs
activity_df = activity_df.assign(perturbation=activity_df.gene + "_" + activity_df.id)

print(activity_df.shape)
activity_df.head()

(128, 8)


,id,sequence,gene,gamma_day5,gamma_day10,relative_activity_day5,relative_activity_day10,perturbation
0,ALDOA_+_30077139.23-P1P2_00,GGTCACCAGGACCCCTTCTG,ALDOA,-0.412746,-0.366469,1.000000,1.000000,ALDOA_ALDOA_+_30077139.23-P1P2_00
1,ALDOA_+_30077139.23-P1P2_06,GGTCACCAGGATCCCTTCTG,ALDOA,-0.396687,-0.348503,0.961091,0.950977,ALDOA_ALDOA_+_30077139.23-P1P2_06
2,ALDOA_+_30077139.23-P1P2_07,GGTCACCAGGCCCCCTTCTG,ALDOA,-0.360892,-0.335059,0.874369,0.914291,ALDOA_ALDOA_+_30077139.23-P1P2_07
3,ALDOA_+_30077139.23-P1P2_13,GGTCACCAGGACCCCTTTTG,ALDOA,0.017063,-0.000220,-0.041340,0.000601,ALDOA_ALDOA_+_30077139.23-P1P2_13
4,ALDOA_+_30077139.23-P1P2_14,GGTCACCAGGACCGCTTCTG,ALDOA,-0.175243,-0.156611,0.424579,0.427353,ALDOA_ALDOA_+_30077139.23-P1P2_14


In [7]:
# Load bulk perturbseq data
bulk_file = pathlib.Path(f"{perturbseq_data_dir}/{gse_id}_bulk_final_analytical.tsv.gz")
bulk_df = pd.read_csv(bulk_file, sep="\t")

# Some genes have very small variance still, remove these!
genes_to_retain = (
    pd.DataFrame(bulk_df.var() > 0.001)
    .reset_index()
    .rename({"index": "gene", 0: "keep"}, axis="columns")
    .query("keep")
    .gene
    .tolist()
)

bulk_subset_df = bulk_df.loc[:, ["Metadata_guide_identity"] + genes_to_retain]

# create a column for the gene
bulk_subset_df = (
    bulk_df
    .assign(Metadata_gene_identity=[x.split("_")[0] for x in bulk_subset_df.Metadata_guide_identity])
    .query("Metadata_gene_identity != '*'")
)

print(bulk_subset_df.shape)
bulk_subset_df.head()

(138, 1002)


/var/folders/41/1jdfcmfs0gx7qwx9d31rtxgr0000gq/T/ipykernel_24789/1102069295.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  pd.DataFrame(bulk_df.var() > 0.001)


,Metadata_guide_identity,Metadata_gene_identity,ABCA1,ABCC3,ABI3BP,AC002331.1,AC002480.3,AC003092.1,AC005616.2,AC006262.5,...,YPEL4,YPEL5,ZBTB38,ZFAS1,ZFP36L1,ZNF365,ZNF43,ZNF483,ZNF556,ZYX
0,ALDOA_ALDOA_+_30077139.23-P1P2_00,ALDOA,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,-0.632468,-0.539091,-0.170790,0.014776,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
1,ALDOA_ALDOA_+_30077139.23-P1P2_06,ALDOA,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,-0.632468,-0.539091,-0.045114,0.042050,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
2,ALDOA_ALDOA_+_30077139.23-P1P2_07,ALDOA,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,-0.632468,-0.539091,-0.101183,0.027887,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
3,ALDOA_ALDOA_+_30077139.23-P1P2_13,ALDOA,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,-0.632468,-0.539091,-0.050848,-0.107389,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229
4,ALDOA_ALDOA_+_30077139.23-P1P2_14,ALDOA,-0.078689,-0.093494,-0.01406,-0.013653,-0.018973,-0.026806,-0.034066,-0.485621,...,-0.09309,-0.632468,-0.539091,0.150701,0.051095,-0.012155,-0.017763,-0.026877,-0.0524,-0.452229


## Calculate Grit

### Bulk profiles

In [8]:
barcode_col = "Metadata_guide_identity"
gene_col = "Metadata_gene_identity"

replicate_group_grit = {
    "profile_col": barcode_col,
    "replicate_group_col": gene_col
}

neg_controls = [x for x in bulk_subset_df.Metadata_guide_identity if "neg_ctrl" in x]
neg_controls

['neg_ctrl_non-targeting_00001',
 'neg_ctrl_non-targeting_00028',
 'neg_ctrl_non-targeting_00054',
 'neg_ctrl_non-targeting_00089',
 'neg_ctrl_non-targeting_00217',
 'neg_ctrl_non-targeting_00283',
 'neg_ctrl_non-targeting_00406',
 'neg_ctrl_non-targeting_00527',
 'neg_ctrl_non-targeting_00802',
 'neg_ctrl_non-targeting_01040']

In [9]:
result = evaluate(
    profiles=bulk_df,
    features=genes_to_retain,
    meta_features=[barcode_col, gene_col],
    replicate_groups=replicate_group_grit,
    operation="grit",
    grit_control_perts=neg_controls
)

result = result.dropna().sort_values(by="grit", ascending=False).reset_index(drop=True)

print(result.shape)
result.head(3)

(138, 3)


,perturbation,group,grit
0,HSPA5_HSPA5_+_128003624.23-P1P2_01,HSPA5,27.978637
1,GATA1_GATA1_-_48645022.23-P1P2_00,GATA1,27.381778
2,HSPA5_HSPA5_+_128003624.23-P1P2_04,HSPA5,27.111261


In [10]:
# Merge with activity results and output file
output_results_file = pathlib.Path(f"results/{gse_id}_grit.tsv")

result = result.merge(activity_df, left_on="perturbation", right_on="perturbation")

result.to_csv(output_results_file, sep="\t", index=False)

print(result.shape)
result.head(3)

(128, 10)


,perturbation,group,grit,id,sequence,gene,gamma_day5,gamma_day10,relative_activity_day5,relative_activity_day10
0,HSPA5_HSPA5_+_128003624.23-P1P2_01,HSPA5,27.978637,HSPA5_+_128003624.23-P1P2_01,GAACCGAGTAGGCGACGGTG,HSPA5,-0.637327,-0.374808,0.852461,0.877397
1,GATA1_GATA1_-_48645022.23-P1P2_00,GATA1,27.381778,GATA1_-_48645022.23-P1P2_00,GTGAGCTTGCCACATCCCCA,GATA1,-0.962732,-0.615306,1.000000,1.000000
2,HSPA5_HSPA5_+_128003624.23-P1P2_04,HSPA5,27.111261,HSPA5_+_128003624.23-P1P2_04,GAGCCGAGAAGGCGACGGTG,HSPA5,-0.754402,-0.422481,1.009055,0.988996


### Single cells

In [11]:
# Determine a proportion of negative control guide population
sc_neg_controls_df = sc_gene_exp_df.query("Metadata_guide_identity in @neg_controls").sample(frac=0.2)

sc_neg_controls = (
    sc_neg_controls_df
    .query("Metadata_guide_identity in @neg_controls")
    .Metadata_cell_identity
    .tolist()
)

replicate_group_grit = {
    "profile_col": "Metadata_cell_identity",
    "replicate_group_col": "Metadata_guide_identity"
}

In [12]:
all_sc_grit_results = []
all_sc_umap_embeddings = []

genes = sc_gene_exp_df.Metadata_gene_identity.unique()
for gene in genes:
    if gene not in ["neg", "*", "nan", np.nan]:
        print(f"Now analyzing {gene}...")
        subset_sc_df = sc_gene_exp_df.query("Metadata_gene_identity in @gene")
        
        # There are a certain number of guides targeting each gene
        guides = subset_sc_df.Metadata_guide_identity.unique()

        # Use the same controls in every experiment
        subset_sc_df = pd.concat([subset_sc_df, sc_neg_controls_df]).reset_index(drop=True)

        # Apply UMAP to single cell profiles (all profiles of one gene + neg controls)
        embedding = umap.UMAP().fit_transform(subset_sc_df.loc[:, genes_to_retain])
        
        # Combine results with single cell dataframe
        embedding_df = pd.concat(
            [
                subset_sc_df.drop(gene_features, axis="columns").reset_index(drop=True),
                pd.DataFrame(embedding, columns=["umap_0", "umap_1"])
            ],
            axis="columns"
        )
        
        # Append to list
        all_sc_umap_embeddings.append(embedding_df.assign(grit_gene=gene))
        
        # Now calculate sc-Grit per guide
        for guide in guides:
            subset_guide_df = pd.concat(
                [
                    subset_sc_df.query("Metadata_guide_identity == @guide"),
                    sc_neg_controls_df
                ]
            ).reset_index(drop=True)
            
            # Calculate Grit
            # Note, every negative control single cell will recieve MULTIPLE grit scores
            # depending on the replicate group information (replicate_group_col)!
            sc_grit_result = evaluate(
                profiles=subset_guide_df,
                features=genes_to_retain,
                meta_features=["Metadata_guide_identity", "Metadata_cell_identity"],
                replicate_groups=replicate_group_grit,
                operation="grit",
                grit_control_perts=[str(x) for x in sc_neg_controls]
            )

            all_sc_grit_results.append(
                sc_grit_result.assign(grit_gene=gene, grit_guide=guide)
            )

Now analyzing RAN...
Now analyzing POLR2H...
Now analyzing TUBB...
Now analyzing CDC23...
Now analyzing POLR1D...
Now analyzing DUT...
Now analyzing HSPA5...
Now analyzing MTOR...
Now analyzing GATA1...
Now analyzing GINS1...
Now analyzing HSPE1...
Now analyzing RPS14...
Now analyzing EIF2S1...
Now analyzing DBR1...
Now analyzing CAD...
Now analyzing SEC61A1...
Now analyzing RPL9...
Now analyzing HSPA9...
Now analyzing RPS18...
Now analyzing ALDOA...
Now analyzing RPS15...
Now analyzing ATP5E...
Now analyzing COX11...
Now analyzing BCR...
Now analyzing GNB2L1...


In [13]:
all_sc_umap_embeddings = pd.concat(all_sc_umap_embeddings).reset_index(drop=True)

# Output file
output_results_file = pathlib.Path(f"results/{gse_id}_single_cell_umap_embeddings.tsv.gz")
all_sc_umap_embeddings.to_csv(output_results_file, sep="\t", compression="gzip", index=False)

print(all_sc_umap_embeddings.shape)
all_sc_umap_embeddings.head()

(32944, 15)


,Metadata_cell_identity,Metadata_cell_barcode,Metadata_guide_identity,Metadata_read_count,Metadata_UMI_count,Metadata_coverage,Metadata_gemgroup,Metadata_good_coverage,Metadata_number_of_cells,Metadata_gene_identity,Metadata_barcode,Metadata_sequence,umap_0,umap_1,grit_gene
0,sc_profile_0,AAACCTGAGAGTAATC-1,RAN_RAN_+_131356438.23-P1P2_12,544.0,34.0,16.000000,1.0,True,1.0,RAN,AAACCTGAGAGTAATC-1,AAACCTGAGAGTAATC,9.562099,2.465169,RAN
1,sc_profile_165,AACGTTGAGAGTAATC-1,RAN_RAN_+_131356438.23-P1P2_00,625.0,36.0,17.361111,1.0,True,2.0,RAN,AACGTTGAGAGTAATC-1,AACGTTGAGAGTAATC,6.261028,0.528125,RAN
2,sc_profile_264,AACTTTCTCTAAGCCA-1,RAN_RAN_+_131356438.23-P1P2_04,334.0,19.0,17.578947,1.0,True,1.0,RAN,AACTTTCTCTAAGCCA-1,AACTTTCTCTAAGCCA,8.225745,1.444978,RAN
3,sc_profile_311,AAGGAGCCATGCGCAC-1,RAN_RAN_+_131356438.23-P1P2_02,405.0,22.0,18.409091,1.0,True,1.0,RAN,AAGGAGCCATGCGCAC-1,AAGGAGCCATGCGCAC,8.999032,3.502961,RAN
4,sc_profile_314,AAGGAGCTCCTGTAGA-1,RAN_RAN_+_131356438.23-P1P2_04,926.0,41.0,22.585366,1.0,True,1.0,RAN,AAGGAGCTCCTGTAGA-1,AAGGAGCTCCTGTAGA,8.748815,0.602991,RAN


In [14]:
all_sc_grit_results = pd.concat(all_sc_grit_results).reset_index(drop=True)

# Output file
output_results_file = pathlib.Path(f"results/{gse_id}_single_cell_grit.tsv.gz")
all_sc_grit_results.to_csv(output_results_file, sep="\t", compression="gzip", index=False)

print(all_sc_grit_results.shape)
all_sc_grit_results.head()

(83105, 5)


,perturbation,group,grit,grit_gene,grit_guide
0,sc_profile_0,RAN_RAN_+_131356438.23-P1P2_12,0.071601,RAN,RAN_RAN_+_131356438.23-P1P2_12
1,sc_profile_10030,RAN_RAN_+_131356438.23-P1P2_12,-0.086802,RAN,RAN_RAN_+_131356438.23-P1P2_12
2,sc_profile_10074,neg_ctrl_non-targeting_00054,0.317055,RAN,RAN_RAN_+_131356438.23-P1P2_12
3,sc_profile_10094,RAN_RAN_+_131356438.23-P1P2_12,0.043757,RAN,RAN_RAN_+_131356438.23-P1P2_12
4,sc_profile_101,neg_ctrl_non-targeting_00406,-0.275052,RAN,RAN_RAN_+_131356438.23-P1P2_12
